In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import pandas_profiling as pp
import matplotlib.pyplot as plt
%matplotlib inline

import gossipcat as gc
import Conversion

import warnings
warnings.filterwarnings('ignore')

wd = '/Users/ewenwang/Documents/practice_data/conversion_rate/'

Data = Conversion.Data(wd, data_list=Conversion.new_data, is_all=False)
dtrain, dvalidation = Data.ValidationFE(validation_day=[25])

In [10]:
def base_counter(data, Base, List, count_base=True, delete_base=True):
    if count_base:
        itemCount = data.groupby([Base], as_index=False)['instance_id'].agg({Base+'_cnt': 'count'})
        data = data.merge(itemCount, on=Base, how='left')

    for f in List:
        itemCount = data.groupby([Base, f], as_index=False)['instance_id'].agg({str(f)+Base+'_cnt': 'count'})
        data = data.merge(itemCount, on=[Base, f], how='left')
        data[str(f)+Base+'_prob'] = data[str(f)+Base+'_cnt'] / data[Base+'_cnt']
    
    if delete_base:
        del data[Base+'_cnt']
    return data

In [12]:
def Item(data):
    item_list = ['item_id', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']
    for i in range(len(item_list)-1):
        data.pipe(base_counter, Base=item_list[i], List=item_list[i+1:])
    return data

In [13]:
temp = Item(dvalidation)

In [14]:
temp.head()

,category_1,category_2,context_id,context_page_id,day,hour,instance_id,is_trade,item_brand_id,item_city_id,...,property_2_cvr,predict_category_0_count,predict_category_0_trade_count,predict_category_0_cvr,predict_category_1_count,predict_category_1_trade_count,predict_category_1_cvr,predict_category_2_count,predict_category_2_trade_count,predict_category_2_cvr
478138,8277336076276184272,,2858926263178884522,4005,25,22,2475218615076601065,NaN,7838285046767229711,7534238860363577544,...,0.017634,97089.0,1264.0,0.013019,36064.0,523.0,0.014502,56410.0,986.0,0.017479
478139,5755694407684602296,,4014843148303402290,4001,25,16,398316874173557226,NaN,8126195666233054089,3948283326616421003,...,0.015676,106334.0,1829.0,0.017201,18961.0,337.0,0.017773,112972.0,2165.0,0.019164
478140,5755694407684602296,,8141706531104182980,4011,25,14,6586402638209028583,NaN,8126195666233054089,3948283326616421003,...,0.015676,106334.0,1829.0,0.017201,18961.0,337.0,0.017773,112972.0,2165.0,0.019164
478141,5755694407684602296,,5878932623083097102,4001,25,15,1040996105851528465,NaN,8126195666233054089,3948283326616421003,...,0.015676,12664.0,281.0,0.022189,48507.0,797.0,0.016431,112972.0,2165.0,0.019164
478142,5755694407684602296,,7218739953079190510,4001,25,15,6316278569655873454,NaN,8126195666233054089,3948283326616421003,...,0.015676,106334.0,1829.0,0.017201,18961.0,337.0,0.017773,11904.0,262.0,0.022009
